## Python script to clean and validate data for fine-tuning

This code was taken from OpenAI's *Data preparation and analysis for chat model fine-tuning* by Michael Wu and Simón Fishman (https://cookbook.openai.com/examples/chat_finetuning_data_prep), and modified for my use case with help from ChatGPT.

In [1]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict
import os

### Loading in data set and validating data 

In [12]:
filename = "finetuning_data_jsonfile.jsonl"
#filename = "finetuning_data_jsonfile2.jsonl"

In [13]:
import os
import json

#data_path = os.path.join("Data", "finetuning_data1.jsonl")
data_path = os.path.join("Data", filename)

# Load the dataset
dataset = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line_number, line in enumerate(f, 1):
        try:
            json_data = json.loads(line)
            dataset.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line {line_number}: {e}")
            print(f"Problematic line content: {line}")


Error decoding JSON on line 1: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)
Problematic line content: {

Error decoding JSON on line 2: Extra data: line 1 column 15 (char 14)
Problematic line content:     "messages": [

Error decoding JSON on line 3: Expecting property name enclosed in double quotes: line 2 column 1 (char 10)
Problematic line content:         {

Error decoding JSON on line 4: Extra data: line 1 column 19 (char 18)
Problematic line content:             "role": "system",

Error decoding JSON on line 5: Extra data: line 1 column 22 (char 21)
Problematic line content:             "content": "Generate quests for a forest-based video game. There are 3 quest types: COLLECT_COINS, TALK_NPC, FIND_OBJECT. For COLLECT_COINS quests, specify NUM_COINS. For TALK_NPC quests, include NPC_NAME and DIALOGUE (one side must be FOX). For FIND_OBJECT quests, provide OBJECT and LOCATION. If the quest rewards coins, add COIN_REWARD. Return a JSON with SERIES_TIT

### Add system instructions 

In [14]:
system_instructions = "Generate quests for a forest-based video game. There are 3 quest types: COLLECT_COINS, TALK_NPC, FIND_OBJECT. For COLLECT_COINS quests, specify NUM_COINS. For TALK_NPC quests, include NPC_NAME and DIALOGUE (one side must be FOX). For FIND_OBJECT quests, provide OBJECT and LOCATION. If the quest rewards coins, add COIN_REWARD. Return a JSON with SERIES_TITLE, SERIES_DESC, and SERIES_STEPS. NPC_NAMES: CHIP, IVY, BEAN; OBJECTS: CUBE_A, BURGER, BANANA, CHEESE, CHERRY, HOTDOG, OLIVE, WATERMELON, BOTTLE; LOCATIONS: BEACH, FOREST, VILLAGE."

### Data

In [20]:
import os
import json

data_path = os.path.join("Data", filename)
print(data_path)
# Load the dataset
dataset = []
with open(data_path, 'r', encoding='utf-8') as f:
    json_string = ""
    for line_number, line in enumerate(f, 1):
        json_string += line.strip()
        try:
            json_data = json.loads(json_string)
            # Modify the content of the third message for each 'assistant' role
            messages = json_data.get("messages", [])
            if len(messages) >= 3 and messages[2].get("role") == "assistant":
                messages[2]["content"] = json.dumps(messages[2]["content"])

            # Modify/add system instructions 
            system_message = next((msg for msg in json_data['messages'] if msg.get("role") == "system"), None)
            if system_message:
                # Modify the existing "Message 1" content for the 'system' role
                system_message["content"] = (system_instructions)
            else:
                # Create a new "Message 1" entry with the role "system"
                json_data['messages'].insert(0, {
                    'role': 'system',
                    'content': (system_instructions)
                })
            
            dataset.append(json_data)
            json_string = ""  # Reset for the next object
        except json.JSONDecodeError:
            #print("DECODE ERROR: " + str(line_number) + ", " + line)
            pass  # Continue reading lines until a complete JSON object is formed

# Initial dataset stats
print("Num examples:", len(dataset))

# Print a subset of the data (e.g., first 5 messages)
print("First 5 messages:")
for i, message in enumerate(dataset[0]["messages"][:5]):
    print(f"Message {i + 1}: {message}")


Data/finetuning_data_jsonfile.jsonl
Num examples: 46
First 5 messages:
Message 1: {'role': 'system', 'content': 'Generate quests for a forest-based video game. There are 3 quest types: COLLECT_COINS, TALK_NPC, FIND_OBJECT. For COLLECT_COINS quests, specify NUM_COINS. For TALK_NPC quests, include NPC_NAME and DIALOGUE (one side must be FOX). For FIND_OBJECT quests, provide OBJECT and LOCATION. If the quest rewards coins, add COIN_REWARD. Return a JSON with SERIES_TITLE, SERIES_DESC, and SERIES_STEPS. NPC_NAMES: CHIP, IVY, BEAN; OBJECTS: CUBE_A, BURGER, BANANA, CHEESE, CHERRY, HOTDOG, OLIVE, WATERMELON, BOTTLE; LOCATIONS: BEACH, FOREST, VILLAGE.'}
Message 2: {'role': 'user', 'content': 'I want to learn how to make delicious baked goods like Chip.'}
Message 3: {'role': 'assistant', 'content': '{"SERIES_TITLE": "Baking Adventure with Chip", "SERIES_DESC": "Join Chip, the friendly bear baker, in a baking adventure where you\'ll learn the art of creating scrumptious baked goods, including hi

In [21]:
from collections import defaultdict

# Assuming dataset is your list of examples loaded from the JSON Lines file

# Format error checks
format_errors = defaultdict(int)

for i, ex in enumerate(dataset, 1):
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for j, message in enumerate(messages, 1):
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
            print(f"Example {i}, Message {j}: Missing 'role' or 'content'")
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
            print(f"Example {i}, Message {j}: Unrecognized key(s): {', '.join(set(message.keys()) - {'role', 'content', 'name', 'function_call'})}")
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            print(f"Example {i}, Message {j}: Unrecognized role '{message.get('role')}'")
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
            print(f"Example {i}, Message {j}: Missing or invalid 'content'")
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1
        print(f"Example {i}: Missing 'assistant' message")

if format_errors:
    print("\nFound errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")


Example 33, Message 3: Missing or invalid 'content'
Example 34, Message 3: Missing or invalid 'content'
Example 35, Message 3: Missing or invalid 'content'
Example 36, Message 3: Missing or invalid 'content'
Example 37, Message 3: Missing or invalid 'content'
Example 38, Message 3: Missing or invalid 'content'
Example 39, Message 3: Missing or invalid 'content'
Example 40, Message 3: Missing or invalid 'content'
Example 41, Message 3: Missing or invalid 'content'
Example 42, Message 3: Missing or invalid 'content'
Example 43, Message 3: Missing or invalid 'content'
Example 44, Message 3: Missing or invalid 'content'
Example 45, Message 3: Missing or invalid 'content'
Example 46, Message 3: Missing or invalid 'content'

Found errors:
missing_content: 14


In [22]:
# Rewrite to a file 

output_path = os.path.join("Data", "modified_dataset2.jsonl")

with open(output_path, 'w', encoding='utf-8') as output_file:
    for example in dataset:
        # Convert the content of the third message back to a string
        messages = example.get("messages", [])
        if len(messages) >= 3 and messages[2].get("role") == "assistant":
            messages[2]["content"] = json.dumps(messages[2]["content"])
        
        # Write the example to the output file
        output_line = json.dumps(example, ensure_ascii=False)
        output_file.write(output_line + '\n')


### Token count 

In [23]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [24]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 376, 958
mean / median: 615.1739130434783, 584.5
p5 / p95: 384.0, 928.5

#### Distribution of num_assistant_tokens_per_example:
min / max: 190, 766
mean / median: 430.8695652173913, 403.5
p5 / p95: 200.5, 738.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [25]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~28298 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~84894 tokens


### Fine-tuning

The following code was modified from the YouTube video *How to Fine-tune a ChatGPT 3.5 Turbo Model - Step by Step Guide* by All About AI (https://youtu.be/2Pd0YExeC5o?si=lB67kd7nr1kj4Fmo&t=500)

In [26]:
def open_file(path):
    with open(path, 'r', encoding='utf-8') as infile:
        return infile.read()

def save_file(path, content):
    with open(path, 'a', encoding='utf-8') as outfile:
        outfile.write(content)

In [27]:
text_obj = open_file("/Users/evahayek/Desktop/Thesis/openAIKeys/openAIKeys.json")
api_keys = json.loads(text_obj)
apiKey = api_keys["apiKey"]
#print(api_keys["apiKey"])

In [28]:
from openai import OpenAI
client = OpenAI(api_key=apiKey)

In [29]:
finetuning_file_path = output_path

client.files.create(
  file=open(finetuning_file_path, "rb"),
  purpose="fine-tune"
)

FileObject(id='file-nlnmrKDSDMFpfU0Wkn0XepYa', bytes=118017, created_at=1700595961, filename='modified_dataset2.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [30]:
#file_id = "file-N43k2LsnVlO4fSIpI0yOK0xv"
file_id = "file-nlnmrKDSDMFpfU0Wkn0XepYa"

response = client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
)

In [31]:
print(response)


FineTuningJob(id='ftjob-KEK7gw5hDSWdZwN0ccKihGP4', created_at=1700596089, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-jbS2rqZp3oeXaCLz0rHKOmjZ', result_files=[], status='validating_files', trained_tokens=None, training_file='file-nlnmrKDSDMFpfU0Wkn0XepYa', validation_file=None)


### FINETUNING JOB INFORMATION + SCREENSHOTS

**IMAGES FOR FINETUNING JOB**

### JOB 1

Files:
![FileJob](Images/fileobject_info.png)


Finetuning Job:
![Finetuning Job](Images/finetuningobj_info.png)

### JOB 2

Files
![FileJob](Images/fileobject2_info.png)

Finetuning Job:
![Finetuning Job](Images/finetuning_info2.png)

